# Context

Earlier in the year, our client, a grocery retailer, ran a campaign to promote their new “Delivery Club” - an initiative that costs a customer 100 dollar per year for membership, but offers free grocery deliveries rather than the normal cost of 10 dollar per delivery.

For the campaign promoting the club, customers were put randomly into three groups - the first group received a low quality, low cost mailer, the second group received a high quality, high cost mailer, and the third group were a control group, receiving no mailer at all.

The client knows that customers who were contacted, signed up for the Delivery Club at a far higher rate than the control group, but now want to understand if there is a significant difference in signup rate between the cheap mailer and the expensive mailer. This will allow them to make more informed decisions in the future, with the overall aim of optimising campaign ROI!



The Chi-Square Test of Independence is typically used when you have categorical data and you want to investigate whether there is a statistically significant association or relationship between two categorical variables.

In our case, you are interested in the number of people who signed up for delivery club aka signup_flag (which is likely a discrete, count variable) and mailer_type (which is categorical - either "mailer1" or "mailer2").

Here's why the Chi-Square Test of Independence is suitable for our hypothesis.

We know that observations are independent, Cells in the contingency table are mutually exclusive, The only thing we need to check is if ~ Expected value of cells should be **5** or greater in at least **80%** of cells.

In [73]:
#Importing the Required libraries

import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency,chi2
from scipy import stats

In [74]:
# Importing the dataset
campaign_data=pd.read_excel(r"/Users/praju/Desktop/DSI/AB_Testing/grocery_database.xlsx",sheet_name='campaign_data')

In [75]:
campaign_data.head()

,customer_id,campaign_name,campaign_date,mailer_type,signup_flag
0,74,delivery_club,2020-07-01,Mailer1,1
1,524,delivery_club,2020-07-01,Mailer1,1
2,607,delivery_club,2020-07-01,Mailer2,1
3,343,delivery_club,2020-07-01,Mailer1,0
4,322,delivery_club,2020-07-01,Mailer2,1


In [76]:
# remove customers who were in the control group
campaign_data = campaign_data.loc[campaign_data['mailer_type']!='Control']

In [77]:
# Calculating Observed_Values

observed_values = pd.crosstab(campaign_data['mailer_type'],campaign_data['signup_flag'])
observed_values

signup_flag,0,1
mailer_type,,
Mailer1,252,123
Mailer2,209,127


In [78]:
# Calculating Signup_rate for mailer1 and mailer2
mailer1_signup_rate = 123/(252+123)
mailer2_signup_rate = 127/(209+127)

print(mailer1_signup_rate)
print(mailer2_signup_rate)

0.328
0.37797619047619047


In [79]:
# Calculating Expected_Values

contingency_table = pd.crosstab(campaign_data['mailer_type'],campaign_data['signup_flag'])

# Calculate expected frequencies
expected_frequencies = stats.contingency.expected_freq(contingency_table)

# Create a DataFrame to display the expected frequencies
expected_df = pd.DataFrame(expected_frequencies, columns=contingency_table.columns, index=contingency_table.index)

# Display the expected frequencies
print("Expected Frequencies:")
print(expected_df)

Expected Frequencies:
signup_flag          0          1
mailer_type                      
Mailer1      243.14346  131.85654
Mailer2      217.85654  118.14346


In [80]:
null_hypothesis = 'There is no relationship between mailer type and signup rate. They are independent'
alternate_hypothesis = 'There is a relationship between mailer type and signup rate. They are not independent'
acceptance_criteria = 0.05

In [81]:
observed_values = pd.crosstab(campaign_data["mailer_type"], campaign_data["signup_flag"]).values

# run the chi-square test
chi2_statistic, p_value, dof, expected_values = chi2_contingency(observed_values, correction = False)

# print chi-square statistic
print(chi2_statistic)


# print p-value
print(p_value)


# find the critical value for our test
critical_value = chi2.ppf(1 - acceptance_criteria, dof)

# print critical value
print(critical_value)


1.9414468614812481
0.16351152223398197
3.841458820694124


In [82]:
# print the results (based upon p-value)
if p_value <= acceptance_criteria:
    print(f"As our p-value of {p_value} is lower than our acceptance_criteria of {acceptance_criteria} - we reject the null hypothesis, and conclude that: {alternate_hypothesis}")
else:
    print(f"As our p-value of {p_value} is higher than our acceptance_criteria of {acceptance_criteria} - we retain the null hypothesis, and conclude that: {null_hypothesis}")


As our p-value of 0.16351152223398197 is higher than our acceptance_criteria of 0.05 - we retain the null hypothesis, and conclude that: There is no relationship between mailer type and signup rate. They are independent


In [83]:
# print the results (based upon chi-square statistic)
if chi2_statistic >= critical_value:
    print(f"As our chi-square statistic of {chi2_statistic} is higher than our critical value of {critical_value} - we reject the null hypothesis, and conclude that: {alternate_hypothesis}")
else:
    print(f"As our chi-square statistic of {chi2_statistic} is lower than our critical value of {critical_value} - we retain the null hypothesis, and conclude that: {null_hypothesis}")
    

As our chi-square statistic of 1.9414468614812481 is lower than our critical value of 3.841458820694124 - we retain the null hypothesis, and conclude that: There is no relationship between mailer type and signup rate. They are independent


## Discussion

While we saw that the higher cost Mailer 2 had a higher signup rate (37.8%) than the lower cost Mailer 1 (32.8%) it appears that this difference is not significant, at least at our Acceptance Criteria of 0.05.

Without running this Hypothesis Test, the client may have concluded that they should always look to go with higher cost mailers - and from what we’ve seen in this test, that may not be a great decision. It would result in them spending more, but not necessarily gaining any extra revenue as a result

Our results here also do not say that there definitely isn’t a difference between the two mailers - we are only advising that we should not make any rigid conclusions at this point.

Running more A/B Tests like this, gathering more data, and then re-running this test may provide us, and the client more insight!